Train and evaluate the convolutional neural network (CNN)
as well as the deep convolutional autoencoder (DCAE)
for the unmixing problem. <br>
Exemplary hyperparameters to set:<br>
- --data-file-path - path to the hyperspectral image (HSI).
- --ground-truth-path - path to the ground truth map containing
the fractions of abundances for entire HSI.
- --train-size - magnitude of the learning set that is utilized
to fine-tune the weights of the model.
- --sub-test-size - size of the test set to evaluate
the generalization of the model. It is sampled from the remaining
HSI excluding the training subset. If not specified, all non-training samples
constitute the test set.<br>Can be employed in the case of experiments
when changing the magnitudes of training sets but keeping the size of test
set constant.
- --n-classes - number of endmembers in the HSI for which the abundances
will be estimated by the model.
- --sample-size - number of spectral bands in a given HSI.
- --neighborhood-size - size of the spatial extent which is employed for each
sample in the form of local neighboring pixels. Most cases allows to leverage
the quality of the segmentation as well as the unmixing.
- --n-runs - number of repetitions of each experiment,
each one has its own seed for the sake of reproducibility.
- --model-name - name of the utilized model, exemplary values:<br>
unmixing_pixel_based_cnn, unmixing_cube_based_cnn, unmixing_pixel_based_dcae,
unmixing_cube_based_dcae for the pixel-based, cube-based CNN and DCAE
respectively.

In [3]:
from os.path import join


base_path = r'../../datasets/urban'
data_file_path = join(base_path, 'urban.npy')
ground_truth_path = join(base_path, 'urban_gt.npy')
endmembers_path = join(base_path, 'urban_m.npy')
train_size = 47000
sub_test_size = 47249
val_size = 0.1
channels_idx = -1
batch_size = 256
shuffle = True
patience = 15
model_name = 'unmixing_cube_based_dcae'
sample_size = 162
neighborhood_size = 5
n_classes = 6
dest_path = base_path
lr = 0.0005
epochs = 2
verbose = 1
n_runs = 2
save_data = False
use_mlflow = False

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from ml_intuition import enums
from ml_intuition.data.utils import parse_train_size, subsample_test_set
from scripts import prepare_data, artifacts_reporter
from scripts.unmixing import train_unmixing, evaluate_unmixing
from scripts.unmixing.unmixing_experiments_runner import NEIGHBORHOOD_SIZES, \
    LEARNING_RATES


for experiment_id in range(n_runs):
    experiment_dest_path = os.path.join(
        dest_path,
        '{}_{}'.format(enums.Experiment.EXPERIMENT, str(experiment_id)))

    os.makedirs(experiment_dest_path, exist_ok=True)

    # Apply default literature hyperparameters:
    if neighborhood_size is None and model_name in NEIGHBORHOOD_SIZES:
        neighborhood_size = NEIGHBORHOOD_SIZES[model_name]
    if lr is None and model_name in LEARNING_RATES:
        lr = LEARNING_RATES[model_name]

    data = prepare_data.main(data_file_path=data_file_path,
                             ground_truth_path=ground_truth_path,
                             train_size=parse_train_size(train_size),
                             val_size=val_size,
                             stratified=False,
                             background_label=-1,
                             channels_idx=channels_idx,
                             neighborhood_size=neighborhood_size,
                             save_data=save_data,
                             seed=experiment_id,
                             use_unmixing=True)
    if sub_test_size is not None:
        subsample_test_set(data[enums.Dataset.TEST], sub_test_size)
    train_unmixing.train(model_name=model_name,
                         dest_path=experiment_dest_path,
                         data=data,
                         sample_size=sample_size,
                         neighborhood_size=neighborhood_size,
                         n_classes=n_classes,
                         lr=lr,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=verbose,
                         shuffle=shuffle,
                         patience=patience,
                         endmembers_path=endmembers_path,
                         seed=experiment_id)

    evaluate_unmixing.evaluate(
        model_path=os.path.join(experiment_dest_path, model_name),
        data=data,
        dest_path=experiment_dest_path,
        neighborhood_size=neighborhood_size,
        batch_size=batch_size,
        endmembers_path=endmembers_path)

    tf.keras.backend.clear_session()

artifacts_reporter.collect_artifacts_report(
    experiments_path=dest_path,
    dest_path=dest_path,
    use_mlflow=use_mlflow)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 3, 3, 160, 16)     448       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 158, 32)     13856     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 156, 64)     6208      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 154, 128)    24704     
_________________________________________________________________
flatten (Flatten)            (None, 19712)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               5046528   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
__________

In [9]:
import pandas as pd

results = pd.read_csv(join(base_path, 'report.csv'))
print(results.iloc[0])

Stats                   mean
aRMSE              0.0642941
aSAM                 0.20149
overallRMSE        0.0944709
rmsAAD              0.336492
perClassSumRMSE     0.549787
class0RMSE          0.101197
class1RMSE          0.114236
class2RMSE         0.0917474
class3RMSE         0.0506221
class4RMSE          0.114342
class5RMSE         0.0776428
rRMSE              0.0600337
rSID                 2196.08
inference_time       31.2769
Name: 0, dtype: object
